In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.3 MB/s eta 0:00:00


In [ ]:
import math
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import GINConv
from torch.nn import Linear, Sequential, ReLU

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

In [ ]:
PARAMS = {
    'hidden_dim': 16,
    'batch_size': 256,
    'epochs': 200,
    'lr': 0.01,
    'weight_decay': 5e-4,
    'seed': 48
}

class Params:
    def __init__(self, obj):
        for k, v in obj.items():
            setattr(self, k, v)

params = Params(PARAMS)

In [ ]:
torch.manual_seed(params.seed)

class GINENet(torch.nn.Module):
    def __init__(self, hidden_dim):
        super(GINENet, self).__init__()
        nn1 = Sequential(Linear(dataset.num_features, hidden_dim))
        self.conv1 = GINConv(nn1)
        nn2 = Sequential(Linear(hidden_dim, hidden_dim))
        self.conv2 = GINConv(nn2)

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GINENet(params.hidden_dim)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
data = data.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=params.lr, weight_decay=params.weight_decay)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_attr)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    logits, accs = model(data.x, data.edge_index, data.edge_attr), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

best_val_acc = 0
best_test_acc = 0
for epoch in range(1, params.epochs):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if val_acc > best_val_acc:
      best_val_acc = val_acc
      best_test_acc = test_acc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
print("Best test acc: ", best_test_acc)

Epoch: 001, Loss: 2.6992, Train Acc: 0.3071, Val Acc: 0.2880, Test Acc: 0.3100
Epoch: 002, Loss: 2.5672, Train Acc: 0.3000, Val Acc: 0.2420, Test Acc: 0.2570
Epoch: 003, Loss: 2.4034, Train Acc: 0.3286, Val Acc: 0.2460, Test Acc: 0.2480
Epoch: 004, Loss: 2.2230, Train Acc: 0.4429, Val Acc: 0.3340, Test Acc: 0.3500
Epoch: 005, Loss: 2.2003, Train Acc: 0.5643, Val Acc: 0.4860, Test Acc: 0.5030
Epoch: 006, Loss: 1.8680, Train Acc: 0.6286, Val Acc: 0.5560, Test Acc: 0.5640
Epoch: 007, Loss: 1.9265, Train Acc: 0.6357, Val Acc: 0.5680, Test Acc: 0.5870
Epoch: 008, Loss: 1.7521, Train Acc: 0.6143, Val Acc: 0.5500, Test Acc: 0.5870
Epoch: 009, Loss: 1.5977, Train Acc: 0.6214, Val Acc: 0.5360, Test Acc: 0.5710
Epoch: 010, Loss: 1.5628, Train Acc: 0.6429, Val Acc: 0.5460, Test Acc: 0.5710
Epoch: 011, Loss: 1.5033, Train Acc: 0.7000, Val Acc: 0.5820, Test Acc: 0.6080
Epoch: 012, Loss: 1.4451, Train Acc: 0.7857, Val Acc: 0.6320, Test Acc: 0.6540
Epoch: 013, Loss: 1.3637, Train Acc: 0.8571, Val Acc